<a href="https://colab.research.google.com/github/camilla-scandola/project-shark-attack/blob/main/quest_sharkatack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install xlrd

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pycountry

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import re

In [6]:
df = pd.read_excel('GSAF5.xls')

In [7]:
pd.set_option('display.max_columns', None)
df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,16th August 2025,2025.0,Provoked,USA,Florida,Cayo Costa Boca Grande,Fishing,Shawn Meuse,M,?,Laceration to right leg below the knee,N,1055 hrs,Lemon shark 1.8 m (6ft),Johannes Marchand: Kevin McMurray Trackingshar...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,18th August,2025.0,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,Brad Ross,M,?,None sustained board severly damaged,N,0730hrs,5m (16.5ft) Great White,Bob Myatt GSAF The Guardian: 9 News: ABS News:...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17th August,2025.0,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,Not stated,M,63,Severe injuries no detail,N,1300hrs,Undetermined,Ralph Collier GSAF and Kevin MCMurray Tracking...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7th August,2025.0,Unprovoked,Australia,NSW,Tathra Beach,Surfing,Bowie Daley,M,9,None sustained board severely damaged,N,1630hrs,Suspected Great White,Bob Myatt GSAF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1st August,2025.0,Unprovoked,Puerto Rico,Carolina,Carolina Beach,Wading,Eleonora Boi,F,39,Bite to thigh area,N,Not stated,Undetermined,Kevin McMurray Trackingsharks.com: NY Post,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#delete empty columns

df = df.drop(columns=[col for col in ["pdf", "href formula", "href", "Case Number", "Case Number.1", "original order", "Unnamed: 21", "Unnamed: 22"] if col in df.columns])

We started analyzing the dataset by country, state, and location, since the first two play a key role in understanding the overall risks of surfing worldwide. For the country column, only 50 entries are missing out of a 7,041 total, so these rows can be dropped without affecting the analysis.
For state and location, however, the number of missing values is much higher. Instead of dropping them (which would remove too much data), we replaced all missing entries with "undefined"

In [9]:
#overview of missing values by country, state, location

missing_country = int(df['Country'].isna().sum())
print(f"Missing country values: {missing_country}")

missing_state = int(df['State'].isna().sum())
print(f"Missing state values: {missing_state}")

missing_location = int(df['Location'].isna().sum())
print(f"Missing location values: {missing_location}")

print(df.shape)

#suggestion: drop all the null values under country (because they won't make a big difference in the analysis, being 50 out of 7041)

df = df.dropna(subset=['Country'])

Missing country values: 50
Missing state values: 485
Missing location values: 567
(7042, 15)


In [10]:
#suggestion: the number of missing values in states and locations is a lot more significant, so I'd replace them with undefined for the time being

df['State'] = df['State'].fillna('undefined')

df['Location'] = df['Location'].fillna('undefined')

We decided to use pycountry to clean the country column. However, we had to be cautious to avoid losing meaningful data. To do this, we created a mapping dictionary to replace the current country names with the standardized ones from pycountry (using AI to help detect all possible variants and mismatches by comparing the initial list of countries with the pycountry-filtered list).

In addition, we filtered out any entries referring to seas or oceans, since these are not valid countries. To ensure we did not discard relevant information, we added code to check whether any of the accidents in rows containing ocean or sea were related to surfing (see below)

In [11]:
#imported pycountry to filter out all entries that do not correspond to standard countries

import pycountry

#cleaning format of Country column by removing any puntuation, extra spaces, and fixing lower-case and capitalization

df['Country_cleaned'] = df['Country'].astype(str).str.strip().str.lower().str.title().str.replace(r'\s+', ' ', regex=True)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    
    print(df['Country_cleaned'].value_counts())

Country_cleaned
Usa                                      2570
Australia                                1507
South Africa                              599
New Zealand                               146
Bahamas                                   141
Papua New Guinea                          136
Brazil                                    122
Mexico                                    107
Italy                                      72
Fiji                                       70
New Caledonia                              66
Philippines                                65
Reunion                                    60
Egypt                                      53
Mozambique                                 50
Cuba                                       49
Spain                                      47
India                                      41
French Polynesia                           39
Japan                                      36
Croatia                                    35
Panama            

The following line of code lists all items containing sea or ocean and their value counts

In [12]:
df[df['Country_cleaned'].str.contains('sea|ocean', case=False, na=False)]['Country_cleaned'].value_counts()

Country_cleaned
Pacific Ocean                            19
Atlantic Ocean                           17
South Atlantic Ocean                     12
Caribbean Sea                             8
Indian Ocean                              7
North Pacific Ocean                       7
Mid Atlantic Ocean                        5
North Atlantic Ocean                      5
St Helena, British Overseas Territory     2
Mediterranean Sea                         2
South Pacific Ocean                       2
Southwest Pacific Ocean                   2
Tasman Sea                                1
Indian Ocean?                             1
Ocean                                     1
Mid-Pacifc Ocean                          1
Coral Sea                                 1
North Sea                                 1
Red Sea / Indian Ocean                    1
British Overseas Territory                1
Red Sea                                   1
South China Sea                           1
Northern Arabian

This code generates a table of accident-related activities in seas and oceans. Since only one entry is related to surfing (is being converted into its respective terrritory in the country_replacements dict), all other ocean and sea records can be safely excluded from the analysis

In [13]:
from IPython.display import display

#filter for sea/ocean and keep both columns

sea_ocean_df = df[df['Country_cleaned'].str.contains('sea|ocean', case=False, na=False)][['Country_cleaned', 'Activity']]

with pd.option_context('display.max_rows', None, 'display.max_columns', None):

    display(sea_ocean_df)

,Country_cleaned,Activity
134,Coral Sea,On a round-the-world expedition
329,British Overseas Territory,NaN
817,"St Helena, British Overseas Territory",Surfing
858,"St Helena, British Overseas Territory",Snorkeling
1158,Atlantic Ocean,Transatlantic Rowing
2218,Atlantic Ocean,Competing in the Woodvale Atlantic Rowing Race
2220,Atlantic Ocean,Competing in the Woodvale Atlantic Rowing Race
2229,Atlantic Ocean,Competing in the Woodvale Atlantic Rowing Race
2685,Caribbean Sea,Sinking of the 40' Esperanza off St. Maartin w...
3043,Northern Arabian Sea,Fell off aircraft carrier


In [14]:
#mapping of common mismatches to ISO pycountry names

country_replacements ={

    # Mixed Country Variants
    "Burma": "Myanmar",
    "Ceylon": "Sri Lanka",
    "Ceylon (Sri Lanka)": "Sri Lanka",
    "Western Samoa": "Samoa",
    "American Samoa": "Samoa",
    "Zaire": "Congo, The Democratic Republic of the",
    "Columbia": "Colombia",
    "Bahrein": "Bahrain",
    "Maldive Islands": "Maldives",
    "Cape Verde": "Cabo Verde",
    "Palestinian Territories": "Palestine, State of",
    "Taiwan": "Taiwan, Province of China",
    "Macau": "Macao", 
    "Okinawa": "Japan",
    "Iran": "Iran, Islamic Republic of",
    "Syria": "Syrian Arab Republic",
    "Korea": "Korea, Republic of",  
    "South Korea": "Korea, Republic of",
    "Usa": "United States",
    "United Arab Emirates (Uae)": "United Arab Emirates",
    "Turkey": "Türkiye",
    "Russia": "Russian Federation",

    # Caribbean & island variations
    "Curacao": "Curaçao",
    "Netherlands Antilles": "Curaçao",
    "Trinidad": "Trinidad and Tobago",
    "Tobago": "Trinidad and Tobago",
    "Trinidad & Tobago": "Trinidad and Tobago",
    "Turks & Caicos": "Turks and Caicos Islands",
    "Turks And Caicos": "Turks and Caicos Islands",
    "Venezuela": "Venezuela",
    "San Domingo": "Dominican Republic",

    # French territories
    "Reunion": "Réunion",
    "Reunion Island": "Réunion",

    # Saint Martin variants
    "St. Martin": "Saint Martin (French part)",
    "St Martin": "Saint Martin (French part)",
    "St. Maartin": "Sint Maarten (Dutch part)",

    # Small states / territories
    "Antigua": "Antigua and Barbuda",
    "Nevis": "Saint Kitts and Nevis",
    "St Kitts / Nevis": "Saint Kitts and Nevis",
    "Falkland Islands": "Falkland Islands (Malvinas)",

    # Papua New Guinea variants
    "British New Guinea": "Papua New Guinea",
    "New Guinea": "Papua New Guinea",
    "New Britain": "Papua New Guinea",

    # Europe
    "England": "United Kingdom",
    "Scotland": "United Kingdom",
    "Crete": "Greece",
    "Canary Islands": "Spain",
    "Azores": "Portugal",

    # Territories / Islands
    "St Helena, British Overseas Territory": "Saint Helena, Ascension and Tristan da Cunha",
    "Johnston Island": "United States",
    "Java": "Indonesia",
}


df["Country_cleaned"] = df["Country_cleaned"].replace(country_replacements)

df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,Country_cleaned
0,16th August 2025,2025.0,Provoked,USA,Florida,Cayo Costa Boca Grande,Fishing,Shawn Meuse,M,?,Laceration to right leg below the knee,N,1055 hrs,Lemon shark 1.8 m (6ft),Johannes Marchand: Kevin McMurray Trackingshar...,United States
1,18th August,2025.0,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,Brad Ross,M,?,None sustained board severly damaged,N,0730hrs,5m (16.5ft) Great White,Bob Myatt GSAF The Guardian: 9 News: ABS News:...,Australia
2,17th August,2025.0,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,Not stated,M,63,Severe injuries no detail,N,1300hrs,Undetermined,Ralph Collier GSAF and Kevin MCMurray Tracking...,Bahamas
3,7th August,2025.0,Unprovoked,Australia,NSW,Tathra Beach,Surfing,Bowie Daley,M,9,None sustained board severely damaged,N,1630hrs,Suspected Great White,Bob Myatt GSAF,Australia
4,1st August,2025.0,Unprovoked,Puerto Rico,Carolina,Carolina Beach,Wading,Eleonora Boi,F,39,Bite to thigh area,N,Not stated,Undetermined,Kevin McMurray Trackingsharks.com: NY Post,Puerto Rico


This code block removes all entries corresponding to vaguely described territories, oceans, and seas

In [15]:
#keeping only entries that match countries in pycountry

pycountry_names = {c.name for c in pycountry.countries}
df = df[df['Country_cleaned'].isin(pycountry_names)]
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df['Country_cleaned'].value_counts())

Country_cleaned
United States                                   2572
Australia                                       1507
South Africa                                     599
Papua New Guinea                                 153
New Zealand                                      146
Bahamas                                          141
Brazil                                           122
Mexico                                           107
Italy                                             72
Fiji                                              70
New Caledonia                                     66
Réunion                                           65
Philippines                                       65
Egypt                                             53
Mozambique                                        50
Cuba                                              49
Spain                                             48
United Kingdom                                    44
Japan                         

In [16]:
#cleaning format of State column by removing any puntuation, extra spaces, and fixing lower-case and capitalization

df['State_cleaned'] = df['State'].astype(str).str.strip().str.lower().str.title().str.replace(r'\s+', ' ', regex=True)

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    
    #print(df['State_cleaned'].value_counts())

In [17]:
state_replacements = {
    # USA typos / spacing
    "Floria": "Florida",
    "Noirth Carolina": "North Carolina",
    
    # Australia typos / spacing
    "Westerm Australia": "Western Australia",
    "Nsw": "New South Wales",
    "New South Ales": "New South Wales",
    "Wa": "West Australia",
    
    # South Africa typos / spacing
    "KNZ": "KwaZulu-Natal",
    "KZN": "KwaZulu-Natal",
    "Easten Cape Province": "Eastern Cape Province",
    
    # Mexico typos
    "Quntana Roo": "Quintana Roo",
    "Quinta Roo": "Quintana Roo",
    "Baja ": "Baja California",   # kept trailing space as abbreviation
    
    # Cuba typos
    "Holquin": "Holguín",
    "Holquin Province": "Holguín Province",
    
    # Caribbean / Bahamas typos
    "New Providence   Isoad": "New Providence",
    "Lucayan Lucayan Archipelago": "Lucayan Archipelago",
    
    # Philippines / islands
    "Guanacoste": "Guanacaste",
    "Queaon": "Quezon",
    "Batanes Provine": "Batanes Province",
    "Lomaiviti Provine": "Lomaiviti Province",
    
    # Saint / St. variants
    "5aint-Denis": "Saint-Denis",
    "St. Georges ": "Saint George’s",
    
    # Unknown placeholder
    "?": "Unknown",
}
df['State_cleaned'] = df['State_cleaned'].replace(state_replacements)

#with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    
    #print(df['State_cleaned'].value_counts())

In [18]:
print(df.shape)

(6825, 17)


In [19]:
#Columns Fatal + Deaths
#checking all unique values in Fatal, to understand how I can better clean it

print(df['Fatal Y/N'].unique())

['N' 'Y' 'F' 'M' nan 'n' 'Nq' 'UNKNOWN' 2017 'Y x 2' ' N' 'N ' 'y']


In [20]:
#cleaning Fatal and printing unique values

df['Fatal Y/N cleaned'] = df['Fatal Y/N'].astype(str).str.strip().str.upper().apply(lambda x: re.sub(r'[^A-Z]', '', x))
print(df['Fatal Y/N cleaned'].unique())

#add column "Death" and assign a numeric value to Fatal, yes or no

deaths_count = {'N': 0, 'Y': 1}
df['Death'] = df['Fatal Y/N cleaned'].map(deaths_count)
deaths_count = int(df['Death'].sum())
print(deaths_count)

['N' 'Y' 'F' 'M' 'NAN' 'NQ' 'UNKNOWN' '' 'YX']
1393


In [21]:
#checking all unique values in Fatal after clean-up

counts = df['Fatal Y/N cleaned'].value_counts(dropna=False)
print(counts)

#all data that is neither Y nor N, is replaced with NaN in the transition from numeric values

counts = df['Death'].value_counts(dropna=False)
print(counts)

Fatal Y/N cleaned
N          4808
Y          1393
NAN         544
UNKNOWN      69
F             5
M             3
NQ            1
              1
YX            1
Name: count, dtype: int64
Death
0.0    4808
1.0    1393
NaN     624
Name: count, dtype: int64


In [22]:
df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,Country_cleaned,State_cleaned,Fatal Y/N cleaned,Death
0,16th August 2025,2025.0,Provoked,USA,Florida,Cayo Costa Boca Grande,Fishing,Shawn Meuse,M,?,Laceration to right leg below the knee,N,1055 hrs,Lemon shark 1.8 m (6ft),Johannes Marchand: Kevin McMurray Trackingshar...,United States,Florida,N,0.0
1,18th August,2025.0,Unprovoked,Australia,NSW,Cabarita Beach,Surfing,Brad Ross,M,?,None sustained board severly damaged,N,0730hrs,5m (16.5ft) Great White,Bob Myatt GSAF The Guardian: 9 News: ABS News:...,Australia,New South Wales,N,0.0
2,17th August,2025.0,Unprovoked,Bahamas,Atlantic Ocean near Big Grand Cay,North of Grand Bahama near Freeport,Spearfishing,Not stated,M,63,Severe injuries no detail,N,1300hrs,Undetermined,Ralph Collier GSAF and Kevin MCMurray Tracking...,Bahamas,Atlantic Ocean Near Big Grand Cay,N,0.0
3,7th August,2025.0,Unprovoked,Australia,NSW,Tathra Beach,Surfing,Bowie Daley,M,9,None sustained board severely damaged,N,1630hrs,Suspected Great White,Bob Myatt GSAF,Australia,New South Wales,N,0.0
4,1st August,2025.0,Unprovoked,Puerto Rico,Carolina,Carolina Beach,Wading,Eleonora Boi,F,39,Bite to thigh area,N,Not stated,Undetermined,Kevin McMurray Trackingsharks.com: NY Post,Puerto Rico,Carolina,N,0.0


In [23]:
# Define search terms related to water sports activities
search_terms = ['surf', 'surfing', 'bodyboard', 'bodyboarding', 'longboard', 'Longboarding', 'windsurf', 'windsurfing', 'kite', 'paddle']
# Filter the dataframe to only include rows where the 'Activity' column contains any of the search terms
# case=False makes the search case-insensitive, na=False skips NA values
df_surf = df[df['Activity'].str.contains('|'.join(search_terms), case=False, na=False)]
# Print the first 20 entries of the 'Activity' column from the filtered dataframe
print(df_surf['Activity'].head(50))

1                                Surfing
3                                Surfing
8                 Surfing (Hydrofoiling)
10                               Surfing
13                               Surfing
20                               Surfing
30                               Surfing
42                               Surfing
46                               Surfing
52                               Surfing
58                               Surfing
60                               Surfing
66                               Surfing
67                               Surfing
76                               Surfing
89                               Surfing
92                               Surfing
99                               Surfing
101                              Surfing
103                              Surfing
104                              Surfing
110                      Paddle boarding
119                              Surfing
120                              Surfing
124             

In [24]:
# Create a new boolean column 'Surfing' that is True if the 'Activity' column contains any of the search terms
# The '|'.join(search_terms) creates a regex pattern with OR operators between each search term
# case=False makes the search case-insensitive, na=False treats NaN values as empty strings
df['Surfing'] = df['Activity'].str.contains('|'.join(search_terms), case=False, na=False)

In [25]:
df['Activity'].value_counts()

Activity
Surfing                                 1134
Swimming                                 979
Fishing                                  485
Spearfishing                             386
Wading                                   177
                                        ... 
Murdered by Thai pirates                   1
Hookah Diving                              1
Sinking of the ferryboat Bongbong 1        1
Sinking of the ferryboat Don Juan          1
Wreck of  large double sailing canoe       1
Name: count, Length: 1504, dtype: int64

In [26]:
#import pycountry
#countries = [c.name for c in pycountry.countries]
#print(countries)

In [27]:
# Extract date patterns from the 'Date' column using regex
# The regex pattern looks for:
# - 4-digit years (e.g., 2023)
# - Numbers with ordinal suffixes (e.g., 1st, 2nd, 3rd, 4th)
# - Date ranges with hyphens (e.g., 1-2, 10-15)
# - Year ranges (e.g., 2020-2022)
# - Month names or other text (e.g., January, Feb)
date_patterns = df['Date'].astype(str).str.extract(r'(\d{4}|\d{1,2}(?:st|nd|rd|th)|\d{1,2}-\d{1,2}|\d{4}-\d{4}|[A-Za-z]+)')

# Display the 13 most common date patterns found in the dataset
print(f"\n THE MOST COMMON TEXT PATTERNS OF 'DATE' COLUMN ARE:\n", date_patterns[0].value_counts().head(13))


 THE MOST COMMON TEXT PATTERNS OF 'DATE' COLUMN ARE:
 0
Jul         664
Aug         596
Reported    532
Sep         520
Jan         494
Jun         476
Oct         443
Apr         434
Dec         431
Mar         416
May         402
Nov         399
Feb         380
Name: count, dtype: int64


In [28]:
# Dictionary mapping various month name formats (including abbreviations and different languages) to standardized 3-letter format
month_mapping = {
    'jan': 'January', 'january': 'January',
    'feb': 'February', 'february': 'Feb', 'fev': 'February',
    'mar': 'March', 'march': 'March',
    'apr': 'April', 'april': 'April', 'abr': 'April',
    'may': 'May',
    'jun': 'June', 'june': 'June',
    'jul': 'July', 'july': 'July',
    'aug': 'August', 'august': 'August', 'ago': 'August',
    'sep': 'September', 'september': 'September', 'set': 'September',
    'oct': 'October', 'october': 'October', 'out': 'October',
    'nov': 'November', 'november': 'November',
    'dec': 'December', 'december': 'December', 'dez': 'December'
}

In [29]:
## Create a regex pattern by joining all month keys with '|' (OR operator)
all_month_patterns = '|'.join(month_mapping.keys())

# Extract month from 'Date' column using regex pattern:
# 1. Convert Date column to string
# 2. Extract month name using case-insensitive regex (?i)
# 3. Convert extracted month to lowercase
# 4. Map to standardized format using the month_mapping dictionary
df['Month'] = df['Date'].astype(str).str.extract(f'(?i)({all_month_patterns})', expand=False).str.lower().map(month_mapping)

In [30]:
# Dictionary mapping numeric month representations to abbreviated month names
numeric_month_mapping = {
    '01': 'January', '02': 'February', '03': 'March', '04': 'April', '05': 'May', '06': 'June',
    '07': 'July', '08': 'August', '09': 'September', '10': 'October', '11': 'November', '12': 'December'
}

# Create a boolean mask for rows where 'Month' column contains NaN values
nan_mask = df['Month'].isna()

# For rows with missing month values, extract the month part (MM) from the 'Date' column
# using regex pattern that matches YYYY-MM-DD format
extracted_numeric_months = df.loc[nan_mask, 'Date'].astype(str).str.extract(r'\d{4}-(\d{2})-\d{2}', expand=False)

# Replace NaN values in 'Month' column with the corresponding month abbreviations
df.loc[nan_mask, 'Month'] = extracted_numeric_months.map(numeric_month_mapping)

In [31]:
# Convert 'Month' column to a categorical data type with ordered months
# This ensures months appear in chronological order rather than alphabetical
df['Month'] = pd.Categorical(df['Month'], categories=['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], ordered=True)

In [32]:
# Display information about the 'Year' column in the dataframe
df['Year'].info()

<class 'pandas.core.series.Series'>
Index: 6825 entries, 0 to 7041
Series name: Year
Non-Null Count  Dtype  
--------------  -----  
6823 non-null   float64
dtypes: float64(1)
memory usage: 106.6 KB


In [33]:
# Inspect the null values in the Year column
df[df['Year'].isnull()]

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,Country_cleaned,State_cleaned,Fatal Y/N cleaned,Death,Surfing,Month
896,Reported 08-Jan-2017,NaN,Invalid,AUSTRALIA,Queensland,undefined,Spearfishing,Kerry Daniel,M,35,"No attack, shark made a threat display",NaN,NaN,Bull shark,Liquid Vision 1/8/2017,Australia,Queensland,NAN,NaN,False,January
6810,Reported 19-Aug-1836,NaN,Unprovoked,ENGLAND,Cumberland,Whitehaven,Swimming,a boy,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",United Kingdom,Cumberland,Y,1.0,False,August


In [34]:
# Fill the two null entries in the 'Year' column by importing the year from the 'Date' column

# Extract the year (4 digits) from the 'Date' column for row 896
year_896 = re.search(r'\d{4}', df.loc[896, 'Date']).group()

# Update the 'Year' column with the extracted year for row 896
df.loc[896, 'Year'] = float(year_896)

df.loc[[896]]

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,Country_cleaned,State_cleaned,Fatal Y/N cleaned,Death,Surfing,Month
896,Reported 08-Jan-2017,2017.0,Invalid,AUSTRALIA,Queensland,undefined,Spearfishing,Kerry Daniel,M,35,"No attack, shark made a threat display",NaN,NaN,Bull shark,Liquid Vision 1/8/2017,Australia,Queensland,NAN,NaN,False,January


In [40]:
# Fill the two null entries in the 'Year' column by importing the year from the 'Date' column

# Extract the year (4 digits) from the 'Date' column for row 6810
year_6810 = re.search(r'\d{4}', df.loc[6810, 'Date']).group()

# Update the 'Year' column with the extracted year for row 6810
df.loc[6810, 'Year'] = float(year_6810)

df.loc[[6810]]

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,Injury,Fatal Y/N,Time,Species,Source,Country_cleaned,State_cleaned,Fatal Y/N cleaned,Death,Surfing,Month
6810,Reported 19-Aug-1836,1836.0,Unprovoked,ENGLAND,Cumberland,Whitehaven,Swimming,a boy,M,NaN,FATAL,Y,NaN,NaN,"C. Moore, GSAF",United Kingdom,Cumberland,Y,1.0,False,August


In [41]:
# Convert the 'Year' column to integer data type for numerical operations

df['Year'] = df['Year'].astype(int)

# Print the data type to verify the conversion was successful
print(df['Year'].dtype)

int64


In [ ]:
#southern hemisphere countries (in our dataset)

southern_countries = [
    "Australia", "Brazil", "South Africa", "Papua New Guinea",
    "New Zealand", "Fiji", "New Caledonia", "Réunion",
    "Indonesia", "Chile", "Argentina", "Uruguay", "Madagascar",
    "Samoa", "Vanuatu", "Tonga", "Kiribati"
]

#assign hemisphere

df["hemisphere"] = df["Country_cleaned"].apply(lambda x: "south" if x in southern_countries else "north")

#map months to seasons for the northern hemisphere

season_map_north = {
    "December": "Winter", "January": "Winter", "February": "Winter",
    "March": "Spring", "April": "Spring", "May": "Spring",
    "June": "Summer", "July": "Summer", "August": "Summer",
    "September": "Autumn", "October": "Autumn", "November": "Autumn"
}

#map months to seasons for the southern hemisphere

season_map_south = {
    "December": "Summer", "January": "Summer", "February": "Summer",
    "March": "Autumn", "April": "Autumn", "May": "Autumn",
    "June": "Winter", "July": "Winter", "August": "Winter",
    "September": "Spring", "October": "Spring", "November": "Spring"
}

#assign seasons

df["season"] = df.apply(
    lambda row: (
        season_map_south.get(row["Month"]) 
        if row["hemisphere"] == "south" 
        else season_map_north.get(row["Month"])
    ),
    axis=1
)

df.head()


In [ ]:
df.to_csv("cleaned_states.csv", index=False, encoding="utf-8")